<a href="https://colab.research.google.com/github/AdrianFernandoPenafiel/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/AG3_Adrian_Penafiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3 - Actividad Guiada 3

Adrian Peñafiel Schweitzer

URL: https://github.com/AdrianFernandoPenafiel/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/AG3_Adrian_Penafiel.ipynb



In [1]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f2c391da860>)

In [2]:
!pip install tsplib95

    100% |████████████████████████████████| 1.6MB 10.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import tsplib95
import random
from math import e

problem = tsplib95.load_problem(file)


#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges()) 

In [0]:
# devuelve el factorial de un numero
def factorial(n):
    if n== 0:
        return 1
    else:
        return n * factorial(n-1)

In [0]:
#Generador de soluciones aleatorias incluyendo el nodo 0
def crear_solucion(Nodos):
    solucion = [0]
    for i in range(len(Nodos)-1):
        solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
    return solucion

In [6]:
crear_solucion(Nodos)

[0,
 22,
 19,
 8,
 9,
 7,
 15,
 40,
 39,
 26,
 36,
 13,
 33,
 11,
 1,
 5,
 29,
 21,
 3,
 4,
 12,
 20,
 23,
 35,
 41,
 28,
 31,
 24,
 16,
 14,
 32,
 25,
 18,
 38,
 37,
 17,
 27,
 6,
 30,
 2,
 34,
 10]

In [0]:
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
    return problem.wfunc(a,b)

In [8]:
distancia(0,2,problem)

30

In [0]:
#sumatoria de aristas
def distancia_total(solucion, problem):
    distancia_total=0
    for i in range(len(solucion)-1):
        distancia_total += distancia(solucion[i], solucion[i+1], problem)
    return distancia_total + distancia(solucion[len(solucion)-1], solucion[0], problem)

In [10]:
distancia_total(crear_solucion(Nodos), problem)

4397

In [0]:
def busqueda_aleatoria(problem, N):
    Nodos = list(problem.get_nodes())
    
    mejor_solucion = []
    mejor_distancia = 10e1000
    
    for i in range(N):
        solucion = crear_solucion(Nodos)
        distancia = distancia_total(solucion, problem)
        
        if distancia < mejor_distancia:
            mejor_solucion = solucion
            mejor_distancia = distancia
    print("Mejor solucion: ", mejor_solucion)
    print("Mejor distancia: ", mejor_distancia)
    return mejor_solucion

In [12]:
sol = busqueda_aleatoria(problem, 800)

Mejor solucion:  [0, 2, 34, 41, 24, 38, 33, 4, 22, 12, 13, 6, 10, 36, 31, 17, 20, 7, 32, 28, 30, 16, 15, 19, 18, 1, 8, 23, 3, 5, 37, 14, 27, 35, 9, 25, 21, 29, 26, 11, 39, 40]
Mejor distancia:  3838


### **Busqueda Local**

In [0]:
def genera_vecina(solucion) :
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

In [14]:
solucion = crear_solucion(Nodos)
print(solucion)
nueva_solucion = genera_vecina(solucion)
print(nueva_solucion)


[0, 12, 20, 22, 23, 13, 14, 30, 24, 18, 19, 4, 6, 9, 32, 41, 17, 36, 3, 38, 25, 16, 21, 10, 29, 11, 27, 2, 37, 34, 35, 8, 31, 33, 28, 15, 1, 7, 26, 39, 40, 5]
[0, 12, 20, 22, 23, 13, 14, 30, 24, 18, 19, 4, 6, 9, 32, 16, 17, 36, 3, 38, 25, 41, 21, 10, 29, 11, 27, 2, 37, 34, 35, 8, 31, 33, 28, 15, 1, 7, 26, 39, 40, 5]


In [0]:
def busqueda_local( problem, N ) :
    mejor_solucion = []
    mejor_distancia = 10e1000
    Nodos = list(problem.get_nodes())
    
    solucion_referencia = crear_solucion(Nodos)
    
    for i in range(N):
        vecina = genera_vecina(solucion_referencia)
        distancia_vecina = distancia_total(vecina, problem)
        
        if distancia_vecina < mejor_distancia :
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
        
        solucion_referencia = vecina
    
    print("Mejor solucion: ", mejor_solucion)
    print("Mejor distancia: ", mejor_distancia)
    return mejor_solucion

In [16]:
sol = busqueda_local(problem, 100)

Mejor solucion:  [0, 29, 9, 23, 41, 25, 10, 8, 28, 2, 4, 12, 11, 18, 17, 31, 35, 36, 37, 16, 19, 13, 5, 26, 6, 1, 14, 15, 7, 30, 21, 40, 24, 39, 22, 38, 34, 33, 20, 32, 27, 3]
Mejor distancia:  1781


In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  return vecina

In [0]:
def probabilidad(T,d):
  r=random.random();
  if(r <= (e**(-1*d)/(T*1.0))):
    return True
  else:
    return False

In [0]:
def bajar_temperatura(T):
    return T-1

In [0]:
def recocido_simulado(problem, TEMPERATURA):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  while TEMPERATURA > 0:
    #Genera una solución vecina(aleatoria)

    #cambiamos la vecina aleatoria
    #vecina = genera_vecina_aleatorio(solucion_referencia)
    vecina = genera_vecina(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    TEMPERATURA = bajar_temperatura(TEMPERATURA)
  
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

In [21]:
sol = recocido_simulado(problem, 100)

La mejor solución encontrada es [0, 1, 6, 26, 18, 12, 11, 25, 10, 3, 4, 5, 13, 19, 14, 16, 15, 37, 7, 32, 34, 30, 28, 2, 27, 17, 36, 35, 31, 29, 8, 41, 23, 9, 40, 24, 21, 39, 22, 38, 33, 20]
con una distancia total de 1686


## **Metaheuristico**

In [0]:
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )

def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [0]:
def hormigas(problem, N):
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N):
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1):
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N):
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

In [25]:
hormigas(problem, 1000)

[0, 18, 13, 25, 29, 32, 6, 2, 28, 5, 30, 12, 20, 3, 31, 7, 37, 16, 27, 34, 15, 1, 33, 26, 41, 10, 8, 11, 23, 4, 19, 17, 39, 24, 40, 38, 21, 22, 35, 36, 14, 9]
3790
